### Scrape PL Data From Multiple Seasons

In [49]:
import time
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd

In [50]:
scraper = cloudscraper.create_scraper()

In [51]:
years = list(range(2025, 2022, -1)) # get previous three seasons
all_matches = [] # store match DFs in a list
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [52]:
for year in years:
    data = scraper.get(standings_url) # get the url for page with PL table
    soup = BeautifulSoup(data.text) # parse through html with bs4 library
    time.sleep(5) # sleep for 5 second to prevent webpage denying access for bot suspicsion 
    standings_table = soup.select('table.stats_table')[0] # grab first table instance on page (table_class.name_of_table)

    # Look for Individual team links in table
    links = standings_table.find_all('a') # a tag defines hyper link


In [53]:
data = scraper.get(standings_url) 
soup = BeautifulSoup(data.text) 
time.sleep(5) 
standings_table = soup.select('table.stats_table')[0]

# Look for Individual team links in table
links = standings_table.find_all('a') 
links

[<a href="/en/squads/822bd0ba/Liverpool-Stats">Liverpool</a>,
 <a href="/en/matches/0a2030a0/Leicester-City-Liverpool-April-20-2025-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>,
 <a href="/en/matches/64bc833f/Liverpool-Tottenham-Hotspur-April-27-2025-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>,
 <a href="/en/matches/157740ee/Chelsea-Liverpool-May-4-2025-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">L</a>,
 <a href="/en/matches/0dc9bdd9/Liverpool-Arsenal-May-11-2025-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">D</a>,
 <a href="/en/matches/1f3db37a/Brighton-and-Hove-Albion-Liverpool-May-19-2025-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">L</a>,
 <a href="/en/players/e342ad68/Mohamed-Salah">Mohamed Salah</a>,
 <a href="/en/players/7a2e46a8/Alisson">Alisson</a>,
 <a